Notebook References:
- xarray access: https://github.com/EOPF-Sample-Service/eopf-sample-notebooks/blob/main/notebooks/Sentinel-2/Sentinel-2_L1C_MSI_Zarr_product_exploration.ipynb
- xcube-eopf: https://eopf-sample-service.github.io/eopf-sample-notebooks/introduction-xcube-eopf-plugin
- xcube-stac: https://github.com/xcube-dev/xcube-stac/blob/main/examples/notebooks/cdse_sentinel_2.ipynb

`conda install xcube-stac xcube-eopf`

In [1]:
import cartopy.crs as ccrs
import numpy as np
#import matplotlib.pyplot as plt
import xarray as xr
#import xarray_eopf
import requests

import dask
from xcube.core.store import new_data_store, get_data_store_params_schema
from xcube_eopf.utils import reproject_bbox
#import xcube
#import xcube_eopf
#import xcube_stac

## 1. Read directly from link

In [2]:
# hamburg notebook returns:

# eopf:
# https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316?.language=de
# https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558?.language=de
# https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250506T103629_N0511_R008_T32UNE_20250506T115207?.language=de

# https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316
# https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558
# https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250506T103629_N0511_R008_T32UNE_20250506T115207

# cdse equivalents: 
# https://browser.stac.dataspace.copernicus.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316?.language=de
# https://browser.stac.dataspace.copernicus.eu/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558?.language=de
# https://browser.stac.dataspace.copernicus.eu/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250506T103629_N0511_R008_T32UNE_20250506T115207?.language=de

# https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316
# https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558
# https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250506T103629_N0511_R008_T32UNE_20250506T115207

In [3]:
# todo: is it possible to access a file directly via xcube?
#path_eopf_zarr = "https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutorial_data/cpm_v253/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458.zarr"
path_eopf_zarr = "https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:202505-s02msil2a/03/products/cpm_v256/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316.zarr"
path_eopf_safe = "" # make available somehow on same bucket as zarr
path_cdse_safe = "s3://eodata/Sentinel-2/MSI/L2A/2025/05/03/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316.SAFE"

## 2. Read via querry
The influence of the underlying STAC API (EOPF vs CDSE) and the implementation of the used software (xcube-stac vs xcube-eopf) affects the performance.

In [4]:
# todo: make dictionary for testing multiple combinations
# ["2025-05-01", "2025-05-07"]
time = ["2025-05-01", "2025-06-01"] # day
time = ["2025-05-01", "2025-05-07"] # month # ideally ["2024-01-01", "2024-02-01"]
time = ["2024-01-01", "2025-01-01"] # year

In [5]:
# choose time 
time = ["2025-05-01", "2025-06-01"]

In [6]:
def create_aoi(bbox, reduction):
    """
    Generate a reduced bounding box or centroid based on a reduction factor.

    Parameters:
    - bbox: [min_lon, min_lat, max_lon, max_lat]
    - reduction: float between 0 and 1
        - 0 returns the centroid as (lon, lat)
        - 0 < reduction < 1 returns a scaled bounding box centered at the centroid

    Returns:
    - centroid tuple or reduced bounding box list
    """
    if not (0 <= reduction <= 1):
        raise ValueError("Reduction must be between 0 and 1.")

    min_lon, min_lat, max_lon, max_lat = bbox

    # Compute centroid
    centroid_lon = (min_lon + max_lon) / 2
    centroid_lat = (min_lat + max_lat) / 2

    #if reduction == 0:
     #   return (centroid_lon, centroid_lat)

    # Compute reduced bounding box dimensions
    lat_span = (max_lat - min_lat) * reduction
    lon_span = (max_lon - min_lon) * reduction

    return [
        centroid_lon - lon_span / 2,
        centroid_lat - lat_span / 2,
        centroid_lon + lon_span / 2,
        centroid_lat + lat_span / 2,
    ]

In [7]:
# pull the bbox from the catalog/object here
url = "https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316"
response = requests.get(url)
item = response.json()
bbox = item["bbox"]

# full scene
print(bbox)
# TO DO: overlap free bbox to make sure only to get the one item we are interested in: maybe item['properties']['geometry']
# quarter scene
print(create_aoi(bbox, 0.25))
# eight scene
print(create_aoi(bbox, 0.125))
# ml patch 256*256
# 10980 x 10980 px; 256/10980
print(create_aoi(bbox, 256/10980))
# pixel
print(create_aoi(bbox, 0))

[8.99969379936479, 53.15557577629945, 10.371024273615161, 54.148104103961266]
[9.51394272720868, 53.52777389917263, 9.856775345771272, 53.77590598108809]
[9.599650881849328, 53.589806919651494, 9.771067191130623, 53.71387296060922]
[9.669372670305636, 53.64026948239441, 9.701345402674315, 53.66341039786631]
[9.685359036489976, 53.65183994013036, 9.685359036489976, 53.65183994013036]


In [8]:
# choose the type of bbox
bbox = create_aoi(bbox, 256/10980)
print(bbox)

[9.669372670305636, 53.64026948239441, 9.701345402674315, 53.66341039786631]


In [9]:
# when accessing via xcube: the crs parameter is mandatory, if crs is different from native crs enforces processing (reprojection, resampling)
crs_utm = item['properties']['proj:code'] # "EPSG:32632"
bbox_utm = reproject_bbox(bbox, "EPSG:4326", crs_utm)
print(bbox_utm)

(544229.9589484755, 5943707.259587298, 546367.9651581453, 5946302.084300316)


In [10]:
# choosing bands with different resolutions enforces processing (resampling)
bands = ["b02"]
# 1
# 2
# all one resolution
# all bands

In [11]:
# everything deviating from native resolution enforces processing  (resampling)
spat_res = 10
# 10
# 20
# 100

In [12]:
# create dictionary for benchmarking to walk over. Add description at which steps processing is enforced. Ideally have some versions without processing.


## STAC EOPF

In [13]:
store_zarr = new_data_store("eopf-zarr")

In [14]:
store_zarr.list_data_ids()

['sentinel-2-l1c', 'sentinel-2-l2a']

In [15]:
store_zarr.get_open_data_params_schema(data_id="sentinel-2-l2a")

In [16]:
%%time
ds_zarr = store_zarr.open_data(
    data_id="sentinel-2-l2a",
    bbox=bbox_utm,
    time_range=time,
    spatial_res=spat_res,
    crs=crs_utm,
    variables=bands,
)
ds_zarr

CPU times: user 3.31 s, sys: 129 ms, total: 3.44 s
Wall time: 12.2 s


<xarray.Dataset> Size: 6MB
Dimensions:      (time: 13, y: 261, x: 215)
Coordinates:
  * time         (time) datetime64[ns] 104B 2025-05-01T10:40:41.025000 ... 20...
    spatial_ref  int64 8B ...
  * x            (x) float64 2kB 5.442e+05 5.442e+05 ... 5.464e+05 5.464e+05
  * y            (y) float64 2kB 5.946e+06 5.946e+06 ... 5.944e+06 5.944e+06
Data variables:
    b02          (time, y, x) float64 6MB dask.array<chunksize=(1, 261, 215), meta=np.ndarray>
Attributes: (4)

## STAC CDSE

In [17]:
credentials = {
    "key": "xxx",
    "secret": "xxx",
}

In [18]:
store_params = get_data_store_params_schema("stac-cdse")
store_params

In [19]:
store_safe = new_data_store("stac-cdse", stack_mode=True, **credentials)

In [20]:
# bands have to be renamed
bands = ['B02']

In [21]:
%%time
ds_safe = store_safe.open_data(
    data_id="sentinel-2-l2a",
    bbox=bbox_utm,
    time_range=time,
    spatial_res=spat_res,
    crs=crs_utm,
    #variables=bands, # --> different names in xcube-cdse and excube-eopf
    asset_names=bands, 
)
ds_safe

CPU times: user 831 ms, sys: 176 ms, total: 1.01 s
Wall time: 12 s


<xarray.Dataset> Size: 4MB
Dimensions:      (time: 16, y: 261, x: 215)
Coordinates:
  * time         (time) datetime64[ns] 128B 2025-05-01T10:40:41.025000 ... 20...
  * x            (x) float64 2kB 5.442e+05 5.442e+05 ... 5.464e+05 5.464e+05
  * y            (y) float64 2kB 5.946e+06 5.946e+06 ... 5.944e+06 5.944e+06
    spatial_ref  int64 8B 0
Data variables:
    B02          (time, y, x) float32 4MB dask.array<chunksize=(1, 261, 215), meta=np.ndarray>
Attributes: (2)

# 